# The Wealth of Nations
# Links Between Economic Health and Environmental Welfare

## 1. Data analysis and collection
Exploration of available databases from World Bank API and selection of indicators

In [4]:
import wbgapi as wb
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import glob


In [ ]:
print(wb.source.info())

economies = list(wb.economy.list())
print(">>>>>>> NUMBER OF ECONOMIES IN THE DATABASE >>>>>>>")
print(len(economies))
print(economies[:5])


Finding the indicators related to the economy such as GDP or GDP growth.

In [ ]:
WDI = 2     # second database = WDI (World Development Indicators)

for ind in wb.series.list(db = WDI, q = ('gdp')):
    print(f"{ind['id']}: {ind['value']}")


Looking over health indicators such as life expectancy, mortality and health expenditure which links economy to health of the population.

In [ ]:
for query in ['life expectancy', 'mortality', 'health expenditure']:
    for ind in wb.series.list(db = WDI, q = query):
        print(f"{ind['id']}: {ind['value']}")


Examining possible indicators for environmental welfare for a country focusing on pollution, use of renewable energy or access to clean water.

In [ ]:
for query in ['CO2', 'pollution', 'greenhouse', 'renewable energy', 'forest', 'water']:
    for ind in wb.series.list(db = WDI, q = query):
        print(f"{ind['id']}: {ind['value']}")


Exploring population its growth and urbanization to see how it affects the economy, health, and environment.

In [ ]:
for query in ['population', 'urban']:
    for ind in wb.series.list(db = WDI, q = query):
        print(f"{ind['id']}: {ind['value']}")

## Choosing main indicators for each group
### Economy 
 - GDP per capita, PPP (constant 2021 international $) — measures average economic output per person.
 - GDP growth (annual %) — tracks how fast the economy is expanding/contracting over time.
 - Carbon intensity of GDP (kg CO2e per 2021 PPP $ of GDP) — shows how much CO2 is emitted per unit of economic output, linking economic activity to environmental cost.

### Health
 - Life expectancy at birth, total (years) — average expected lifespan at birth, measure of overall health.
 - Mortality rate, under-5 (per 1,000 live births) — deaths of children under five, demonstrates healthcare access and quality.
 - Current health expenditure (% of GDP) — investment in health sector.

-- Indicators which I found under environment indicators explorations but are related to health sector and will provide meaningful links between these sectors, connects environmental conditions to human health.
 - Mortality rate attributed to household and ambient air pollution, age-standardized (per 100,000 population)connects environmental conditions to human health.
 - Mortality rate attributed to unsafe water, unsafe sanitation and lack of hygiene (per 100,000 population)

### Environment
 - Carbon dioxide (CO2) emissions (total) excluding LULUCF (Mt CO2e)
 - Carbon dioxide (CO2) emissions excluding LULUCF per capita (t CO2e/capita)CO2 per capita 
 - PM2.5 air pollution, mean annual exposure (micrograms per cubic meter) — indicator of air quality.
 - Total greenhouse gas emissions excluding LULUCF (Mt CO2e)
 - Total greenhouse gas emissions excluding LULUCF per capita (t CO2e/capita)
 - Renewable energy consumption (% of total) — sustainability metric.
 - Forest area (% of land area) —  air quality, biodiversity.
 - People using safely managed drinking water services (% of population) — access to clean drinking water.

### Population and Urbanization Indicators
 - Population growth (annual %)
 - Population, total
 - Urban population (% of total population) — share of people living in cities, reflecting urbanization level.
 - Population density (people per sq. km of land area) — influencing environmental stress.

Next steps: selecting the countries, pulling the data, checking the missing values, possibly adjusting number of indicators based on coverage.

## Country selection
1. Selecting the real countries - filtering out economies in the list named such as Africa Eastern and Southern, World...
2. Keeping the sample of countries limited to larger economies - filtering out small countries with population under 5 millions

In [ ]:
for e in wb.economy.list(): 
    if e['aggregate'] == False:
        print(f"{e['id']}: {e['value']}")

In [ ]:
non_aggregates = [e for e in wb.economy.list() if e['aggregate'] == False]
print(len(non_aggregates))
economy_ids = [e['id'] for e in non_aggregates]

# Filtering economies with population > 5 million
min_pop = 5000000  # minimum population
pop_data = wb.data.DataFrame('SP.POP.TOTL', economy_ids)
print(pop_data[:3])

# Selecting countries satisfying the population criterion for the last available year
filtered_countries = pop_data.index[pop_data.iloc[:, -1] >= min_pop].tolist()
print(filtered_countries[:10])
print(len(filtered_countries))


After filtering, I am left with 126 countries. This set is sufficient to capture global trends while reducing the number of very small economies, which helps save computing time. In the data_collection.py I collect all the data for chosen indicators and countries.

In [ ]:
df = pd.read_csv("../data/raw/all_indicators.csv")
print(df.head())
print(df.shape)


In [ ]:
year_cols = df.columns[2:]

# Sort indicators by missing fraction in descending order
missing_per_indicator = df.groupby('series')[year_cols].apply(lambda x: x.isna().mean().mean())
print("Missing values per indicator:")
print(missing_per_indicator.sort_values(ascending=False))

The indicators that have more than 30% of the data missing are removed. And new data frame is created.

In [ ]:
low_data_indicators = missing_per_indicator[missing_per_indicator > 0.3].index
df_clean1 = df[~df['series'].isin(low_data_indicators)]

Countries are checked next and ones with over 30% of missing data removed

In [ ]:
missing_per_country = df_clean1.groupby('economy')[year_cols].apply(lambda x: x.isna().mean().mean())
print("Missing values per country:")
print(missing_per_country.sort_values(ascending = False))
low_data_countries = missing_per_country[missing_per_country > 0.3].index
df_clean2 = df_clean1[~df_clean1['economy'].isin(low_data_countries)]
df_clean2.to_csv('data/cleaned/all_indicators_cleaned.csv', index=False)

Saved the cleaned file and checking remaining indicators to set names for them which will be used for their separate dataframes.

In [ ]:
remaining_indicators = df_clean2['series'].unique()
print(remaining_indicators)

indicator_names = {
            'AG.LND.FRST.ZS': 'forest_area',
            'EG.FEC.RNEW.ZS': 'renewable_energy',
            'EN.ATM.PM25.MC.M3': 'pm25_pollution',
            'EN.GHG.ALL.PC.CE.AR5': 'ghg_per_capita',
            'EN.GHG.CO2.PC.CE.AR5': 'co2_per_capita',
            'EN.GHG.CO2.RT.GDP.PP.KD': 'carbon_intensity',
            'NY.GDP.MKTP.KD.ZG': 'gdp_growth',
            'NY.GDP.PCAP.PP.KD': 'gdp_per_capita',
            'SH.DYN.MORT': 'child_mortality',
            'SH.XPD.CHEX.GD.ZS': 'health_exp_pct_gdp',
            'SH.XPD.CHEX.PC.CD': 'health_exp_per_capita',
            'SP.DYN.LE00.IN': 'life_expectancy',
            'SP.POP.GROW': 'population_growth',
            'SP.POP.TOTL': 'population',
            'SP.URB.TOTL.IN.ZS': 'urban_population'
        }


In data_cleaning.py this and more is provided to remove the lowest coverage indicators and countries and then the cleaned dataframe is split into several csv files one per each indicator using the functions clean_data and split_data which will be executed in the main.py

## Analysis of the final dataframes

In [8]:
import os
print("Current working directory:", os.getcwd())

Current working directory: /Users/jarka/Desktop/Python-project/exploration_and_analysis


In [22]:
# Get list of all CSV files in the cleaned folder
csv_files = glob.glob("../data/cleaned/*.csv")
print(csv_files)

# Load each CSV into a dictionary of DataFrames
data = {}
for f in csv_files:
    name = os.path.splitext(os.path.basename(f))[0]  
    # splitext splits name and extension [0] to choose filename without extension
    # basename gets the filename from the full path
    data[name] = pd.read_csv(f)
    
print(list(data.keys()))


['../data/cleaned/forest_area.csv', '../data/cleaned/child_mortality.csv', '../data/cleaned/renewable_energy.csv', '../data/cleaned/all_indicators_cleaned.csv', '../data/cleaned/population_growth.csv', '../data/cleaned/co2_per_capita.csv', '../data/cleaned/health_exp_per_capita.csv', '../data/cleaned/pm25_pollution.csv', '../data/cleaned/population.csv', '../data/cleaned/ghg_per_capita.csv', '../data/cleaned/gdp_per_capita.csv', '../data/cleaned/carbon_intensity.csv', '../data/cleaned/life_expectancy.csv', '../data/cleaned/health_exp_pct_gdp.csv', '../data/cleaned/urban_population.csv', '../data/cleaned/gdp_growth.csv']
['forest_area', 'child_mortality', 'renewable_energy', 'all_indicators_cleaned', 'population_growth', 'co2_per_capita', 'health_exp_per_capita', 'pm25_pollution', 'population', 'ghg_per_capita', 'gdp_per_capita', 'carbon_intensity', 'life_expectancy', 'health_exp_pct_gdp', 'urban_population', 'gdp_growth']


### Plotting

In [23]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.scatterplot(
    data=data['gdp_per_capita'].merge(data['life_expectancy'], on='economy'),
    x='gdp_per_capita',
    y='life_expectancy'
)
plt.title("GDP per Capita vs Life Expectancy")
plt.show()

ValueError: Could not interpret value `gdp_per_capita` for `x`. An entry with this name does not appear in `data`.